In [1]:
import os
import numpy as np
import pandas as pd
import time
import torch
from copy import deepcopy
import torch.nn as nn
# from dataloader import *
from torch.utils.data import Dataset, DataLoader, RandomSampler
import argparse, os
# from modules import attmil,clam,mhim,dsmil,transmil,mean_max
from modules import transmil, attmil, dsmil, mhim
from torch.nn.functional import one_hot
from torch.cuda.amp import GradScaler
from contextlib import suppress
import time

from timm.utils import AverageMeter, dispatch_clip_grad
from timm.models import  model_parameters
from collections import OrderedDict

from utils import *

from sklearn.metrics import roc_auc_score
# from tqdm.notebook import tqdm
from tqdm import tqdm
# import cv2
# import torch.nn.functional as F
# from torchvision import transforms
# from torch.utils.data import Dataset, DataLoader
# from torch.optim.lr_scheduler import _LRScheduler
# from torchvision.transforms.functional import to_pil_image
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from PIL import Image
import torchstain
import warnings
import pickle
import gc
import random
from datetime import datetime

/home/admin/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
warnings.filterwarnings(action='ignore')
gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model = mhim.MHIM(select_mask=False, n_classes=args.n_classes, act=args.act, head=args.n_heads, da_act=args.da_act, baseline=args.baseline).to(device)
model = mhim.MHIM(select_mask=False, n_classes=2, act='relu', head=8, da_act='relu', baseline='selfattn').to(device)
model_tea = deepcopy(model)

# _str = 'fold_{fold}_model_best_auc.pt'.format(fold=args.fold_num) # fold=k
_str = 'fold_{fold}_model_best_auc.pt'.format(fold=2) # fold=k
_teacher_init = os.path.join('/data/notebook/hyena/modules/init_ckp/c16_3fold_init_transmil_seed2021',_str)

# pre_dict = torch.load(_teacher_init)
# # print(pre_dict)
# info = model_tea.load_state_dict(pre_dict,strict=False)

In [7]:
# import argparse

# parser = argparse.ArgumentParser(description='MIL Training Script')
# parser.add_argument('--model', default='mhim', type=str, help='Model name') # mhim
# parser.add_argument('--no_tea_init', action='store_true', help='Without teacher initialization')
# parser.add_argument('--init_stu_type', default='none', type=str, help='Student initialization [none, fc, all]') # none
# parser.add_argument('--tea_type', default='same', type=str, help='[none, same]') # none
# parser.add_argument('--no_log', action='store_true', help='Without log') ## store_true

model_type = 'mhim'
no_tea_init = 'False'
init_stu_type = 'none'
tea_type = 'same'
no_log = 'False'

# args = parser.parse_args()

if init_stu_type != 'none':
    if not no_log:
        print('######### Model Initializing.....')
    pre_dict = torch.load(_teacher_init)
    new_state_dict ={}
    if init_stu_type == 'fc':
    # only patch_to_emb
        for _k, v in pre_dict.items():
            _k = _k.replace('patch_to_emb.','') if 'patch_to_emb' in _k else _k
            new_state_dict[_k]=v
        info = model.patch_to_emb.load_state_dict(new_state_dict,strict=False)
    else:
    # init all
        info = model.load_state_dict(pre_dict,strict=False)
    if not no_log:
        print(info)

# teacher model
if model_type == 'mhim':
    model_tea = deepcopy(model)
    if not no_tea_init and tea_type != 'same':
        if not no_log:
            print('######### Teacher Initializing.....')
            pre_dict = torch.load(_teacher_init)
            # print(pre_dict)
            info = model_tea.load_state_dict(pre_dict,strict=False)
        try:
            pre_dict = torch.load(_teacher_init)
            info = model_tea.load_state_dict(pre_dict,strict=False)
            if not no_log:
                print(info)
        except:
            if not no_log:
                print('########## Init Error')
    if tea_type == 'same':
        model_tea = model
else:
    model_tea = None

In [10]:
print(model_tea)

MHIM(
  (dp): Dropout(p=0.25, inplace=False)
  (patch_to_emb): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
  )
  (online_encoder): SAttention(
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (layer1): TransLayer(
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): NystromAttention(
        (to_qkv): Linear(in_features=512, out_features=1536, bias=False)
        (to_out): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): Dropout(p=0.1, inplace=False)
        )
        (res_conv): Conv2d(8, 8, kernel_size=(33, 1), stride=(1, 1), padding=(16, 0), groups=8, bias=False)
      )
    )
    (layer2): TransLayer(
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): NystromAttention(
        (to_qkv): Linear(in_features=512, out_features=1536, bias=False)
        (to_out): Sequential(
          (0): Linear(in_features=512, ou